# Testeo RAG Texto plano con BGE chunkc=1000 y overlap=80%

USAR ENVIRONMENT local_llm_26042926

In [1]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

In [5]:
local_llm = "llama3.2:latest"
#url_llm = "http://172.17.30.133:11434" # ordenados con 2 gpus
#url_llm = "http://172.17.30.172:11434" #COMPUTACION
url_llm = "http://localhost:11434"
from langchain_community.llms import Ollama
llm = Ollama(model=local_llm, base_url=url_llm, temperature=0) 

In [6]:
# SE USA UNA DE LAS GPU
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"  # En lugar de bge-large-en
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [7]:
from langchain.vectorstores import Chroma
vectorstore_load = Chroma(persist_directory="./chroma_db_1000_bge_rag_texto", embedding_function= emb)

C:\Users\emolt\AppData\Local\Temp\ipykernel_3604\2885107945.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_load = Chroma(persist_directory="./chroma_db_1000_bge_rag_texto", embedding_function= emb)


In [8]:
retriever = vectorstore_load.as_retriever(search_type="similarity", search_kwargs={"k": 8})

In [9]:
from langchain import hub
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks and an expert in research projects funded by the European Union under the Horizon 2020 programme.
Use the following context from Horizon 2020 projects to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context} 

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

### Factuales

In [11]:
retrieved_docs_1 = retriever.get_relevant_documents(query_1)
answer_1 = rag_chain.invoke(query_1) #"What is the objective of the project with grant agreement 740934?"
answer_1

C:\Users\emolt\AppData\Local\Temp\ipykernel_3604\75204582.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs_1 = retriever.get_relevant_documents(query_1)


"I don't have information on a specific Horizon 2020 project with grant agreement 740934. Without more context or details, I'm unable to provide an accurate answer regarding its objectives. If you could provide more information about the project, I'd be happy to try and assist further."

ES CORRECTAAA!

In [12]:
retrieved_docs_2 = retriever.get_relevant_documents(query_2)
answer_2 = rag_chain.invoke(query_2) #"What is the total cost of the project with the acronym HYPERGRYD?"
answer_2

"I don't have information on a Horizon 2020 project with the acronym HYPERGRYD, including its total cost. I couldn't find any relevant data in my knowledge base. If you need more specific information, please provide more context or details about the project."

NO DA RESPUESTA

In [13]:
retrieved_docs_3 = retriever.get_relevant_documents(query_3)
answer_3 = rag_chain.invoke(query_3) #"How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
answer_3

'I couldn\'t find any information on a Horizon 2020 project with the title "Transforming Research through Innovative Practices for Linked interdisciplinary Exploration". I don\'t know the specific funding allocation for this project. If you have more context or details, I may be able to help further.'

INCORRECTA

In [14]:
retrieved_docs_4 = retriever.get_relevant_documents(query_4)
answer_4 = rag_chain.invoke(query_4) # "Which organisation played the role of coordinator in the grant agreement 777998?"
answer_4

"I don't have information on the specific Horizon 2020 project with grant agreement 777998, including the organization that played the role of coordinator. I can try to search for more information or provide general information on Horizon 2020 projects if needed. If you have any additional context or details about the project, I may be able to help further."

NO DA RESPUESTA

In [15]:
retrieved_docs_5 = retriever.get_relevant_documents(query_5)
answer_5 = rag_chain.invoke(query_5) # "What topic does the project with the acronym INTERRFACE belong to?"
answer_5

"I don't know the specific topic of the Horizon 2020 project with the acronym INTERRFACE. However, I can suggest that it might be related to a research project in the field of environmental or social sciences. If you need more information, please provide more context or details about the project."

NO DA RESPUESTA

In [16]:
retrieved_docs_6 = retriever.get_relevant_documents(query_6)
answer_6 = rag_chain.invoke(query_6) # "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
answer_6

'I\'m not aware of any specific information about a Horizon 2020 project titled "European Joint Programme on Radioactive Waste Management". However, I can suggest that it\'s possible that this project was framed under the Interregional Cooperation (I) legal basis. If you need more information, please provide more context or clarify which project you are referring to.'

CORRECTA

In [17]:
retrieved_docs_7 = retriever.get_relevant_documents(query_7)
answer_7 = rag_chain.invoke(query_7) # "What type of proposal was the grant agreement 814416?"
answer_7

"I don't have information on a specific Horizon 2020 project with grant agreement number 814416. I can try to search for more context or details if you provide it. Alternatively, I can suggest checking the EU's Horizon 2020 database or contacting the European Commission directly for more information."

NO DA RESPUESTA

In [18]:
retrieved_docs_8 = retriever.get_relevant_documents(query_8)
answer_8 = rag_chain.invoke(query_8) # "To which master call was the project with the acronym G9NIGHT submitted?"
answer_8

'I don\'t know the specific master call to which the Horizon 2020 project "G9NIGHT" was submitted. However, I can suggest checking the official EU Horizon 2020 website or the project\'s documentation for more information. If you need further assistance, please provide more context or details about the project.'

NO DA RESPUESTA

In [19]:
retrieved_docs_9 = retriever.get_relevant_documents(query_9)
answer_9 = rag_chain.invoke(query_9) # "To which sub call was the project titled Electron Nanocrystallography submitted?"
answer_9

'I don\'t know the specific sub-call to which the project "Electron Nanocrystallography" was submitted. However, I can suggest checking the official Horizon 2020 project database or contacting the European Commission for more information. The project\'s details might be available on the EU Research & Innovation website.'

CORRECTA

### Sobre proyectos y temáticas

In [20]:
retrieved_docs_10 = retriever.get_relevant_documents(query_10)
answer_10 = rag_chain.invoke(query_10) # "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
answer_10

"I'm not aware of a specific Horizon 2020 project with a grant agreement available for public access. However, I can suggest that you check the EU's Horizon 2020 database or contact the European Commission directly for more information on AI-related projects. If you provide more context or details about the objective, I may be able to help further."

CORRECTA COMPLETAMENTE

In [26]:
retrieved_docs_11 = retriever.get_relevant_documents(query_11)
answer_11 = rag_chain.invoke(query_11) # "Provide the acronym of 1 project which objective is related to robotics."
answer_11

'I\'m not aware of a specific Horizon 2020 project with an objective directly related to robotics. However, I can suggest that you look into the "Robotics for Energy and Environmental Sustainability" (ROB-4-EES) project, which is part of the Horizon 2020 programme.'

CORRECTA

In [36]:
retrieved_docs_12 = retriever.get_relevant_documents(query_12)
answer_12 = rag_chain.invoke(query_12) # "Provide the title of 1 project which objective is related to geolocation."
answer_12

'I\'m not aware of a specific Horizon 2020 project with an objective directly related to geolocation. However, I can suggest that you check the EU\'s Horizon 2020 database for projects under the "Geospatial" or "Location-based Services" theme. If you need more information, please provide more context or details about the type of geolocation-related objective you\'re looking for.'

CORRECTA ENTERA

In [37]:
retrieved_docs_13 = retriever.get_relevant_documents(query_13)
answer_13 = rag_chain.invoke(query_13) # "Provide the objective of 1 project related to digital twin."
answer_13

'I\'m not aware of a specific Horizon 2020 project directly related to digital twin objectives. However, I can suggest that the "Digital Twin for Industry" project (H2020-ICT-2016-2) aimed to develop and demonstrate digital twins in various industries such as manufacturing, energy, and transportation. The project\'s objective was to create a common framework for the development of digital twins across different sectors.'

NO ES CORRECTA, EL ACRONIMO LLEVA TWIN PERO NO DESCRIBE UN DIGITAL TWIN SINO QUE el El objetivo principal mencionado es aumentar la capacidad de investigación del NVRI en colaboración con instituciones de Alemania y Dinamarca, enfocándose en enfermedades infecciosas animales, zoonosis y seguridad alimentaria

In [39]:
retrieved_docs_14 = retriever.get_relevant_documents(query_14)
answer_14 = rag_chain.invoke(query_14) # "Provide the objective of 3 different projects related to corrosion."
answer_14

'I can provide information on Horizon 2020 projects related to corrosion. \n\n1. The project "CORROD" aimed to develop new methods for predicting and preventing corrosion in industrial applications, focusing on advanced materials and coatings.\n2. The project "CORROTECT" focused on developing novel protective coatings for steel structures exposed to corrosive environments, with a focus on sustainability and environmental impact.\n3. I don\'t know the specific objective of the third Horizon 2020 project related to corrosion.'

TODA LA INFORMACIÓN ES CORRECTA

In [40]:
retrieved_docs_15 = retriever.get_relevant_documents(query_15)
answer_15 = rag_chain.invoke(query_15) # "Provide the title of 3 different projects which objective is related to offshore structures."
answer_15

'I can provide some examples of Horizon 2020 projects with objectives related to offshore structures. Three such projects are:\n\n1. "OceanGate" - a project focused on developing and demonstrating new technologies for the construction, installation, and maintenance of offshore wind farms.\n2. "DeepSeaNet" - a project aimed at improving the design, construction, and operation of deepwater offshore platforms.\n3. "Offshore Renewable Energy (ORE) Demonstrator" - a project that focuses on the development and demonstration of new technologies for the production, transmission, and distribution of renewable energy from offshore wind farms.\n\nPlease note that I may not have exhaustive information on all Horizon 2020 projects, but these are some examples related to offshore structures.'

CORRECTA TOTALMENTE AUNQUE NO DA LOS TITULOS, QUE ES LO QUE SE PIDE

In [41]:
retrieved_docs_16 = retriever.get_relevant_documents(query_16)
answer_16 = rag_chain.invoke(query_16) # "Provide the acronym of 3 different projects which objective is related to materials engineering."
answer_16

'I can provide some examples of Horizon 2020 projects related to materials engineering. Three such projects are:\n\n1. NMP-01-2017-NMP31-0374 - "NanoMaterials for Energy Applications"\n2. FETOPEN-03-2018-010956 - "Graphene and 2D Materials for Energy Storage and Conversion"\n3. SUSTAIN-04-2015-028911 - "Sustainable materials and technologies for the circular economy"\n\nPlease note that I may not have information on all Horizon 2020 projects, especially those with limited or no publicly available information.'

CORRECTA TOTALMENTE

In [42]:
retrieved_docs_17 = retriever.get_relevant_documents(query_17)
answer_17 = rag_chain.invoke(query_17) # "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
answer_17

'I don\'t have access to specific grant agreements for individual Horizon 2020 projects. However, I can provide general information on some EU-funded projects that focus on nanocomposites.\n\n1. Project: "NanoComposites" (Grant Agreement No.: 289334) - This project aimed to develop new nanocomposite materials with improved mechanical and thermal properties.\n2. Project: "NANOCOMPOSITES FOR ENERGY APPLICATIONS" (Grant Agreement No.: 764372) - This project focused on the development of nanocomposite materials for energy-related applications, such as solar cells and fuel cells.\n3. Project: "NanoComposites for Advanced Biomaterials" (Grant Agreement No.: 777747) - This project explored the use of nanocomposites in advanced biomaterials for medical and dental applications.\n\nPlease note that I don\'t have direct access to the grant agreements, and this information is based on publicly available general information.'

TODO ES CORRECTO, SALVO EL TITULO DEL SEGUNDO PROYECTO, que en vez del titulo es una descripcion del proyecto

### Preguntas sobre socios

In [43]:
retrieved_docs_18 = retriever.get_relevant_documents(query_18)
answer_18 = rag_chain.invoke(query_18) # "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
answer_18

'I can provide information on Horizon 2020 projects related to artificial intelligence. One example is the project "Human Brain Project" (HBP), which aimed to develop a computational model of the human brain and its functions, with applications in AI and neuroscience. Another example is the project "AI for Robotics" (AI4ROB), which focused on developing AI technologies for robotics and automation.'

COMPROBAR SI ES CORRECTA, ESTA EMPRESA HA PARTICIPADO EN: 871177 y 952176

In [44]:
retrieved_docs_19 = retriever.get_relevant_documents(query_19)
answer_19 = rag_chain.invoke(query_19) # "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
answer_19

'I\'m not aware of a specific organisation with activity type PRC (Pre-Commercial Procurement) that has participated in Horizon 2020 projects related to robotics. However, I can suggest some EU-funded projects that involve robotics and PRC, such as the "Robotics for Energy Access" project or the "Smart Robotics for Industry 4.0" project. If you need more information, please provide more context or details.'

TODAS SON PRC, CHECKEAR TEMATICA, AUNQUE PARECE QUE ESTA BIEN

In [45]:
retrieved_docs_20 = retriever.get_relevant_documents(query_20)
answer_20 = rag_chain.invoke(query_20) # "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
answer_20

"I'm not able to provide specific information on individual organisations, but I can tell you that several SMEs have participated in Horizon 2020 projects focused on geolocation. For example, companies like Garmin, u-blox, and Trimble have been involved in projects related to geospatial technologies and location-based services. If you need more specific information, please let me know and I'll try to help further."

PIC, NOMBRE Y SME CORRECTO. CHECKEAR TEMATICA AUNQUE PARECE CORRECTO

In [46]:
retrieved_docs_21 = retriever.get_relevant_documents(query_21)
answer_21 = rag_chain.invoke(query_21) # "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
answer_21

"I'm not aware of a specific EU Horizon 2020 project with a direct connection to digital twins. However, I can suggest that the European Union's Digital Single Market (DSM) initiative has funded several projects related to Industry 4.0 and digitalization, which may include aspects of digital twin technology. If you're looking for more information, I recommend searching the EU Horizon 2020 project database or contacting a relevant EU agency directly."

CHECKEAR

In [47]:
retrieved_docs_22 = retriever.get_relevant_documents(query_22)
answer_22 = rag_chain.invoke(query_22)
answer_22 # "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."

"I don't have information on specific organisations participating in Horizon 2020 projects related to corrosion. However, I can suggest some ways for you to find the required information. You can search the EU's Horizon 2020 database or contact the European Commission directly for more details."

LA INFORMACION ES CORRECTA, CHEKCERA LA TEMATICA DE SUS PROYECTOS

In [48]:
retrieved_docs_23 = retriever.get_relevant_documents(query_23)
answer_23 = rag_chain.invoke(query_23)
answer_23 # "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."

"I'm not aware of a specific European organization that has participated in Horizon 2020 projects with an objective directly related to offshore structures. However, I can suggest some organizations that have been involved in marine-related projects under Horizon 2020, such as the European Marine Observation Network (EMONET) or the Ocean Observatories Initiative (OOI). If you need more information, please provide more context or clarify your question."

NO DA RESPUESTA

In [49]:
retrieved_docs_24 = retriever.get_relevant_documents(query_24)
answer_24 = rag_chain.invoke(query_24)
answer_24 # "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."

"I don't have access to real-time information on specific European SMEs participating in Horizon 2020 projects. However, I can suggest some general resources where you may be able to find the information you're looking for, such as the EU's Horizon 2020 website or the Enterprise Europe Network. If you provide more context or details, I'd be happy to try and assist you further."

TOTALMENTE CORRECTA

In [50]:
retrieved_docs_25 = retriever.get_relevant_documents(query_25)
answer_25 = rag_chain.invoke(query_25)
answer_25 # "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

'I\'m not aware of a specific European SME that has played the role of coordinator in Horizon 2020 projects focused on nanocomposites. However, I can suggest some EU-funded projects that involve nanocomposites, such as "Nanomaterials for Energy Storage" or "NanoComposites for Advanced Materials". If you need more information, please provide more context or details about the project you are looking for.'

NO DA RESPUESTA

In [51]:
len(retrieved_docs_25)

0

### Se guardan los resultados en un excel

In [33]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------------------- -------------------- 5.8/11.6 MB 29.4 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 29.1 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------

In [34]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
df['Consulta'] = [query_1, query_2, query_3, query_4, query_5, 
                  query_6, query_7, query_8, query_9, query_10, 
                  query_11, query_12, query_13, query_14, query_15, 
                  query_16, query_17,query_18, query_19, query_20, 
                  query_21, query_22, query_23, query_24, query_25]

df['Retrieved_context_1'] = [retrieved_docs_1[0], retrieved_docs_2[0], retrieved_docs_3[0], retrieved_docs_4[0], retrieved_docs_5[0], 
                             retrieved_docs_6[0], retrieved_docs_7[0], retrieved_docs_8[0], retrieved_docs_9[0], retrieved_docs_10[0], 
                             retrieved_docs_11[0], retrieved_docs_12[0], retrieved_docs_13[0], retrieved_docs_14[0], retrieved_docs_15[0], 
                             retrieved_docs_16[0], retrieved_docs_17[0], retrieved_docs_18[0], retrieved_docs_19[0], retrieved_docs_20[0], 
                             retrieved_docs_21[0], retrieved_docs_22[0], retrieved_docs_23[0], retrieved_docs_24[0], retrieved_docs_25[0]]

df['Retrieved_context_2'] = [
    retrieved_docs_1[1], retrieved_docs_2[1], retrieved_docs_3[1], retrieved_docs_4[1],
    retrieved_docs_5[1], retrieved_docs_6[1], retrieved_docs_7[1], retrieved_docs_8[1],
    retrieved_docs_9[1], retrieved_docs_10[1], retrieved_docs_11[1], retrieved_docs_12[1],
    retrieved_docs_13[1], retrieved_docs_14[1], retrieved_docs_15[1], retrieved_docs_16[1],
    retrieved_docs_17[1], retrieved_docs_18[1], retrieved_docs_19[1], retrieved_docs_20[1],
    retrieved_docs_21[1], retrieved_docs_22[1], retrieved_docs_23[1], retrieved_docs_24[1],
    retrieved_docs_25[1]
]

df['Retrieved_context_3'] = [
    retrieved_docs_1[2], retrieved_docs_2[2], retrieved_docs_3[2], retrieved_docs_4[2],
    retrieved_docs_5[2], retrieved_docs_6[2], retrieved_docs_7[2], retrieved_docs_8[2],
    retrieved_docs_9[2], retrieved_docs_10[2], retrieved_docs_11[2], retrieved_docs_12[2],
    retrieved_docs_13[2], retrieved_docs_14[2], retrieved_docs_15[2], retrieved_docs_16[2],
    retrieved_docs_17[2], retrieved_docs_18[2], retrieved_docs_19[2], retrieved_docs_20[2],
    retrieved_docs_21[2], retrieved_docs_22[2], retrieved_docs_23[2], retrieved_docs_24[2],
    retrieved_docs_25[2]
]

df['Retrieved_context_4'] = [
    retrieved_docs_1[3], retrieved_docs_2[3], retrieved_docs_3[3], retrieved_docs_4[3],
    retrieved_docs_5[3], retrieved_docs_6[3], retrieved_docs_7[3], retrieved_docs_8[3],
    retrieved_docs_9[3], retrieved_docs_10[3], retrieved_docs_11[3], retrieved_docs_12[3],
    retrieved_docs_13[3], retrieved_docs_14[3], retrieved_docs_15[3], retrieved_docs_16[3],
    retrieved_docs_17[3], retrieved_docs_18[3], retrieved_docs_19[3], retrieved_docs_20[3],
    retrieved_docs_21[3], retrieved_docs_22[3], retrieved_docs_23[3], retrieved_docs_24[3],
    retrieved_docs_25[3]
]

df['Retrieved_context_5'] = [
    retrieved_docs_1[4], retrieved_docs_2[4], retrieved_docs_3[4], retrieved_docs_4[4],
    retrieved_docs_5[4], retrieved_docs_6[4], retrieved_docs_7[4], retrieved_docs_8[4],
    retrieved_docs_9[4], retrieved_docs_10[4], retrieved_docs_11[4], retrieved_docs_12[4],
    retrieved_docs_13[4], retrieved_docs_14[4], retrieved_docs_15[4], retrieved_docs_16[4],
    retrieved_docs_17[4], retrieved_docs_18[4], retrieved_docs_19[4], retrieved_docs_20[4],
    retrieved_docs_21[4], retrieved_docs_22[4], retrieved_docs_23[4], retrieved_docs_24[4],
    retrieved_docs_25[4]
]

df['Retrieved_context_6'] = [
    retrieved_docs_1[5], retrieved_docs_2[5], retrieved_docs_3[5], retrieved_docs_4[5],
    retrieved_docs_5[5], retrieved_docs_6[5], retrieved_docs_7[5], retrieved_docs_8[5],
    retrieved_docs_9[5], retrieved_docs_10[5], retrieved_docs_11[5], retrieved_docs_12[5],
    retrieved_docs_13[5], retrieved_docs_14[5], retrieved_docs_15[5], retrieved_docs_16[5],
    retrieved_docs_17[5], retrieved_docs_18[5], retrieved_docs_19[5], retrieved_docs_20[5],
    retrieved_docs_21[5], retrieved_docs_22[5], retrieved_docs_23[5], retrieved_docs_24[5],
    retrieved_docs_25[5]
]

df['Retrieved_context_7'] = [
    retrieved_docs_1[6], retrieved_docs_2[6], retrieved_docs_3[6], retrieved_docs_4[6],
    retrieved_docs_5[6], retrieved_docs_6[6], retrieved_docs_7[6], retrieved_docs_8[6],
    retrieved_docs_9[6], retrieved_docs_10[6], retrieved_docs_11[6], retrieved_docs_12[6],
    retrieved_docs_13[6], retrieved_docs_14[6], retrieved_docs_15[6], retrieved_docs_16[6],
    retrieved_docs_17[6], retrieved_docs_18[6], retrieved_docs_19[6], retrieved_docs_20[6],
    retrieved_docs_21[6], retrieved_docs_22[6], retrieved_docs_23[6], retrieved_docs_24[6],
    retrieved_docs_25[6]
]

df['Retrieved_context_8'] = [
    retrieved_docs_1[7], retrieved_docs_2[7], retrieved_docs_3[7], retrieved_docs_4[7],
    retrieved_docs_5[7], retrieved_docs_6[7], retrieved_docs_7[7], retrieved_docs_8[7],
    retrieved_docs_9[7], retrieved_docs_10[7], retrieved_docs_11[7], retrieved_docs_12[7],
    retrieved_docs_13[7], retrieved_docs_14[7], retrieved_docs_15[7], retrieved_docs_16[7],
    retrieved_docs_17[7], retrieved_docs_18[7], retrieved_docs_19[7], retrieved_docs_20[7],
    retrieved_docs_21[7], retrieved_docs_22[7], retrieved_docs_23[7], retrieved_docs_24[7],
    retrieved_docs_25[7]
]


df['Answer'] = [answer_1, answer_2, answer_3, answer_4, answer_5,
    answer_6, answer_7, answer_8, answer_9, answer_10, 
    answer_11, answer_12, answer_13, answer_14, answer_15,
    answer_16, answer_17, answer_18, answer_19, answer_20, 
    answer_21, answer_22, answer_23, answer_24, answer_25
]


df

IndexError: list index out of range

In [47]:
!pip install openpyxl

  Using cached openpyxl-3.1.4-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.4-py2.py3-none-any.whl (251 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [35]:
df.to_excel('Testeo_RAG_Texto_1000_bge_18072024.xlsx', index=False)